In [ ]:
import arcpy

# First Raster
# Detect Objects

arcpy.env.processorType = "GPU"

in_raster_1 = r"path_to_first_raster"
out_detected_objects_1 = "path_to_new_shape_file_after_detection"
in_model_definition = "path_to_model"
model_arguments = "padding 2; threshold 0.75; batch_size 3; tile_size 768; return_bboxes False"
run_nms = "NMS"
confidence_score_field = "Confidence"
class_value_field = "Class"
max_overlap_ratio = 0
processing_mode = "PROCESS_AS_MOSAICKED_IMAGE"

arcpy.CheckOutExtension("ImageAnalyst")

arcpy.ia.DetectObjectsUsingDeepLearning(in_raster_1, out_detected_objects_1,
   in_model_definition, model_arguments, run_nms, confidence_score_field,
   class_value_field, max_overlap_ratio, processing_mode)

# Regularize Building Footprints

arcpy.env.processorType = "CPU"

Input_Features_RBF_1 = out_detected_objects_1
Output_Feature_Class_RBF_1 = "path_to_new_shape_file_after_RBF"
Method = "RIGHT_ANGLES"
Tolerance = 1
Precision = 0.25

arcpy.RegularizeBuildingFootprint_3d(Input_Features_RBF_1, Output_Feature_Class_RBF_1, Method, Tolerance, Precision)

# Change Polygons to Lines

arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS_1989_Poland_CS92")

Input_Features_P2L_1 = Output_Feature_Class_RBF_1
Output_Feature_Class_P2L_1 = "path_to_new_shape_file_after_P2L"
arcpy.PolygonToLine_management(Input_Features_P2L_1, Output_Feature_Class_P2L_1, "IDENTIFY_NEIGHBORS")

# Second Raster
# Detect Objects

arcpy.env.processorType = "GPU"

in_raster_2 = r"path_to_second_raster"
out_detected_objects_2 = "path_to_new_shape_file_after_detection"

arcpy.CheckOutExtension("ImageAnalyst")

arcpy.ia.DetectObjectsUsingDeepLearning(in_raster_2, out_detected_objects_2,
   in_model_definition, model_arguments, run_nms, confidence_score_field,
   class_value_field, max_overlap_ratio, processing_mode)

# Regularize Building Footprints

arcpy.env.processorType = "CPU"

Input_Features_RBF_2 = out_detected_objects_2
Output_Feature_Class_RBF_2 = "path_to_new_shape_file_after_RBF"

arcpy.RegularizeBuildingFootprint_3d(Input_Features_RBF_2, Output_Feature_Class_RBF_2, Method, Tolerance, Precision)

# Change Polygons to Lines

arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS_1989_Poland_CS92")

Input_Features_P2L_2 = Output_Feature_Class_RBF_2
Output_Feature_Class_P2L_2 = "path_to_new_shape_file_after_P2L"
arcpy.PolygonToLine_management(Input_Features_P2L_2, Output_Feature_Class_P2L_2, "IDENTIFY_NEIGHBORS")

# Detect Feature Changes

Lines2021 = Output_Feature_Class_P2L_1
Lines2020 = Output_Feature_Class_P2L_2

arcpy.DetectFeatureChanges_management(Lines2021, Lines2020, r"path_to_new_comparison_shape_file", 5)

# Creating new layer file with new buildings only

shp_path = r"path_to_comparison_shape_file"
arcpy.MakeFeatureLayer_management(shp_path, "new_layer_name", "CHANGE_TYP = 'N'")
arcpy.SaveToLayerFile_management("layer_name", r"path_to_new_layer_file")